In [353]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [144]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [145]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [146]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [147]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [148]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [149]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [150]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

# print(demand_array)
# VoT_array_base

In [151]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], 5))

T = 5
for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [152]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [153]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [154]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [155]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [156]:
# def coeff_from_input(coeff_input, num_edges, num_gp_lanes):
#     # Goal: Differentiate between express and general purpose lanes.
    
#     assert np.all(coeff_input >= 0.0), "All entries of coeff_input must be non-negative."
#     assert len(coeff_input.shape) == 2, "coeff_input should be a matrix, i.e., 2-D array"
#     assert coeff_input.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
#     assert coeff_input.shape[1] == num_edges, "Latency functions should be defined across all edges."
    
#     coeff_length = 3
    
#     ex_to_gp_multiplier = np.array([1, 1/num_gp_lanes, num_gp_lanes]).reshape((coeff_length, 1)) \
#                             @ np.ones((1, num_edges))
    
#     coeff_full = np.zeros((coeff_length, num_edges, 2))
#     coeff_full[:, :, 0] = coeff_input
#     coeff_full[:, :, 1] = coeff_input * ex_to_gp_multiplier
    
#     return coeff_full

def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
    print()
    print("in_indices:", in_indices)
    print("el_indices:", el_indices)
    print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g, t] * ell[e, 0, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [157]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [158]:
def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0, B_min = 0.1):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
#                     for od in range(len(od_to_edges_list_full))]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    B_max_list = [sum(tau_feas.value[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                            for od in range(len(od_to_edges_list_full))]
#     B_max_wrt_tau_feas = min([B_max for B_max in B_max_list if B_max >= B_min])
    B_max_wrt_tau_feas = max(B_max_list)
    B_feas = min(B, B_max_wrt_tau_feas)
    
    print()
    print("B:", B)
    print("B_max_wrt_tau_feas:", B_max_wrt_tau_feas)
    print("B_feas:", B_feas)
    print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas, 0.0), decimals=4)
    
#     return tau_feas.value, B_feas



In [159]:
# # OLD def proj_tau_B:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0):

# #     print()
# #     print("tau.shape[0]:", tau.shape[0])
# #     print("num_edges:", num_edges)
# #     print()

#     assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
#     tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
#     func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

#     objective = cp.Minimize(func)

#     constraints = []
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od]) \
#                     for od in range(len(od_to_edges_list_full)) for t in range(T)]
    
#     prob = cp.Problem(objective, constraints)
#     result = prob.solve()

# #     print()
# #     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
# #     print()

#     return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
#             np.round(np.maximum(B_feas.value, 0.0), decimals=4)

# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [160]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [161]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edge_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edge_to_od)

In [162]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input):
    
#     print("tau (in solve_CBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for t in range(T):
            func += coeff[0, e] * x[e, 0, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 0, t] - coeff[2, e], 0))
            func += coeff[0, e] * x[e, 1, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("B:", B)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                    <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## <font color='blue'>Implement Zeroth-order Gradient Descent:</font> 

In [163]:
## Functions defined above:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max = 1.0, B_max = 1.0):

# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

In [164]:
# coeff_input
# demand_array.shape

In [322]:

T = 5

# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3


num_el = 2
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

# B = 0.8
a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


# TODO: Modify coeff_input:

## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)
B_upper_limit = max([sum(tau_upper_limit[e, t] for e in od_to_edges_list_full[od] for t in range(T) ) \
                     for od in range(len(od_to_edges_list_full))])

print("tau_upper_limit:", tau_upper_limit)
print()
print("B_upper_limit:", B_upper_limit)

d = num_edges * T + 1

tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)

fraction_tau_max = 0.5
fraction_B_max_1 = 0.25
fraction_B_max_2 = 0.5
assert fraction_B_max_1 < fraction_B_max_2, "We must have fraction_B_max_1 < fraction_B_max_2."

# tau[:, :, 0] = fraction_tau_max * tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)

# B[0] = fraction_B_max * B_upper_limit
# B_upper_limit_init = max([sum(tau[e, t, 0] for e in od_to_edges_list_full[od] for t in range(T) ) \
#                      for od in range(len(od_to_edges_list_full))])
B_upper_limit_init = sum(tau[e, t, 0] for e in range(num_edges) for t in range(T) )

B[0] = np.random.triangular(0.0, fraction_B_max_1 * B_upper_limit_init, fraction_B_max_2 * B_upper_limit_init)

print()
print("tau[:, :, 0]:", tau[:, :, 0])
print()
print("np.sum(tau[:, :, 0]):", np.sum(tau[:, :, 0]))
print()
print("B_upper_limit_init:", B_upper_limit_init)
print()
print("B[0]:", B[0])

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# B[0] = B_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

tau_upper_limit: [[0.31942364 0.31942364 0.31942364 0.31942364 0.31942364]
 [0.24983105 0.24983105 0.24983105 0.24983105 0.24983105]
 [0.99953381 0.99953381 0.99953381 0.99953381 0.99953381]
 [1.02805916 1.02805916 1.02805916 1.02805916 1.02805916]
 [1.6042613  1.6042613  1.6042613  1.6042613  1.6042613 ]
 [0.19224693 0.19224693 0.19224693 0.19224693 0.19224693]
 [0.21782897 0.21782897 0.21782897 0.21782897 0.21782897]]

B_upper_limit: 23.055924301525522

tau[:, :, 0]: [[0.17423042 0.16307276 0.15094248 0.17645771 0.10536026]
 [0.13820058 0.08063373 0.19068939 0.12448639 0.10586947]
 [0.73633324 0.69066432 0.18555649 0.37150008 0.49964696]
 [0.87963292 0.74921522 0.18787907 0.19483651 0.75439165]
 [0.76442224 0.91970929 0.55218721 0.97110016 0.97314223]
 [0.07113572 0.14115204 0.08265377 0.13253633 0.07555474]
 [0.06898136 0.12155195 0.15884319 0.14731105 0.07886958]]

np.sum(tau[:, :, 0]): 11.918750495276756

B_upper_limit_init: 11.918750495276754

B[0]: 3.9151395720150175


In [323]:
time_1 = time.time()

welfare_list = []

## Set lambda:

lambda_E, lambda_R, lambda_I = 2.0, 5.0, 5.0

delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 1E-5
delta_bar = 0.01

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(d)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("B[i], before projection:\n", B[i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], before projection:\n", B_perturbed[i])

    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, num_edges, tau_perturbed[:, :, i], B_perturbed[i], \
                                                        od_to_edges_list_full, \
                                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("B[i], after projection:\n", B[i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], after projection:\n", B_perturbed[i])

    # TODO: Remove "network":

    y_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], B[i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input)

    y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((num_edges, T)) \
                        * (welfare_perturbed - welfare)
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("B[i+1], before projection:\n", B[i+1])
    
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, num_edges, tau[:, :, i+1], B[i+1], od_to_edges_list_full, \
                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("B[i+1], after projection:\n", B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i]), axis = 2)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:\n", tau_diffs)
        print("B_diffs:\n", B_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            print("Within error bound.")
            break

    time_2 = time.time()

    print()
    print("Iteration count:", i)
    print("Time:", time_2 - time_1)
    print()




Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[0.17423042 0.16307276 0.15094248 0.17645771 0.10536026]
 [0.13820058 0.08063373 0.19068939 0.12448639 0.10586947]
 [0.73633324 0.69066432 0.18555649 0.37150008 0.49964696]
 [0.87963292 0.74921522 0.18787907 0.19483651 0.75439165]
 [0.76442224 0.91970929 0.55218721 0.97110016 0.97314223]
 [0.07113572 0.14115204 0.08265377 0.13253633 0.07555474]
 [0.06898136 0.12155195 0.15884319 0.14731105 0.07886958]]
B[i], before projection:
 3.9151395720150175
tau_perturbed[:, :, i], before projection:
 [[0.17455375 0.16337227 0.15146568 0.17682291 0.10464286]
 [0.13789193 0.08073349 0.19071643 0.12457489 0.1060962 ]
 [0.73638945 0.69051703 0.18597723 0.37116001 0.49954161]
 [0.87985441 0.74922213 0.18801814 0.19497539 0.75451741]
 [0.76430189 0.9196966  0.55213628 0.97146329 0.97335848]
 [0.07174638 0.14153873 0.08268912 0.13271012 0.07534466]
 [0.06869321 0.12127121 0.15884355 0.14724004 0.07907781]]
B_perturbed[i], before projection:
 3.9148941128


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3886298.8603505925
obj_E: 5970.350535707792
obj_R: 4821.915115157867
obj_I: 779693.5469709933

welfare_perturbed: 3934742.729308542
obj_E_perturbed: 5968.796136139301
obj_R_perturbed: 4824.740463534965
obj_I_perturbed: 789385.7678707876

tau[:, :, i+1], before projection:
 [[ -30.17085087  -42.14408585   42.7382242    48.57297972   -7.28171864]
 [ -30.71799623  -25.21499565   47.9282107   -36.03906385   -5.92015357]
 [ -19.75767908   14.28522787   49.46583847   70.05235162  -43.94091741]
 [ -11.42284917   -0.12064005   17.59940036  -11.07445247    8.23835052]
 [ -14.01250175   10.92765283   33.80347938   14.52842881 -105.82978715]
 [   8.1546787    59.70732594   -5.03906818  -16.51276137  -13.64389375]
 [ -35.91653681   -6.89616793  -38.96315045  -37.80206394   54.50081932]]
B[i+1], before projection:
 46.89951333351204

B: 46.89951333351204
B_max_w


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3701366.942238591
obj_E: 6289.696311164387
obj_R: 9127.708211156503
obj_I: 746885.218134409

welfare_perturbed: 3726080.049229669
obj_E_perturbed: 6302.600261270793
obj_R_perturbed: 9128.160833137686
obj_I_perturbed: 751823.1305745632

tau[:, :, i+1], before projection:
 [[-16.31857993   4.69165805  -8.36657658  -9.3558901   32.81022886]
 [ -8.92192065   3.34645718 -12.50049832  -2.4749448   29.02209591]
 [ 14.34361355  -5.11806569   3.79977184  48.0204709  -15.30481257]
 [-15.80147943 -19.06113128   1.68476485  10.87860047   9.03121797]
 [ -1.45686072  20.15231106   5.91107141  -7.67306645 -14.73692193]
 [ 14.54923861 -13.20403629   4.83222433   5.51857672   3.3425424 ]
 [  2.35850479  32.27026761  17.67504609  -9.57932287   9.96789954]]
B[i+1], before projection:
 -6.712762952159744

B: -6.712762952159744
B_max_wrt_tau_feas: 12.070114496551142
B_f


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3817694.169933118
obj_E: 6220.176193609883
obj_R: 8490.14178876381
obj_I: 769540.9052979435

welfare_perturbed: 3831003.8248084527
obj_E_perturbed: 6218.2408891214645
obj_R_perturbed: 8491.737275879774
obj_I_perturbed: 772205.2058819218

tau[:, :, i+1], before projection:
 [[ -6.7074595    8.81316276   0.8286062    7.85180183  10.38939152]
 [  1.52635572  13.68572836   0.62480212   5.07413222 -10.87872498]
 [  3.63848067  -4.45250545  -1.99583396   7.61013009   0.88946121]
 [ -3.34437727   7.2001863  -13.1389549    4.78296679  15.83995151]
 [  6.36333557   1.14634085  10.56758002  -3.3417611   -7.60341692]
 [  0.38222311  -0.24884261  -6.62843719  -9.96431146  -3.5282515 ]
 [  4.6510213  -13.67826485  12.2072912   -8.99577101   2.03899265]]
B[i+1], before projection:
 12.175275711079719

B: 12.175275711079719
B_max_wrt_tau_feas: 13.450322350872618
B


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3733733.6318831844
obj_E: 6135.648888009583
obj_R: 6756.510184036455
obj_I: 751048.9770054695

welfare_perturbed: 3797246.967686816
obj_E_perturbed: 6140.9393376018215
obj_R_perturbed: 6757.024103433816
obj_I_perturbed: 763750.0419057563

tau[:, :, i+1], before projection:
 [[ 76.63442298  -6.64944082  33.49425016  25.53886129 -28.94888497]
 [ 18.37870641  -9.26209337  15.2841251  -40.91974833  -1.58226734]
 [-25.98740896  20.92035472  -6.4463669   32.79380239 -15.89092335]
 [-19.09500025 -43.79922623 -29.45444711  33.86092837   2.37599662]
 [-24.13262424 -32.92926152  84.11719595  54.44284908  10.7448133 ]
 [ -5.4952538  -14.91181266 -14.60865102  34.42809234 -12.17767005]
 [ 81.71084683  39.26252136 -46.08253327  23.19829463   7.23936969]]
B[i+1], before projection:
 -6.951152278829715

B: -6.951152278829715
B_max_wrt_tau_feas: 11.3894656722903
B_

...


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3709624.0686299135
obj_E: 6118.473946052401
obj_R: 4557.037998192235
obj_I: 744034.4621457539

welfare_perturbed: 3752479.265995495
obj_E_perturbed: 6121.345523106002
obj_R_perturbed: 4557.971430792148
obj_I_perturbed: 752605.2864206487

tau[:, :, i+1], before projection:
 [[ -9.82774225  -8.59967868  18.57884792   4.67417504  -9.60768296]
 [ 16.27554855   4.41834734   5.98503796  -7.44668589   8.67599171]
 [  2.32978072 -10.12341322  -1.51023432   9.44593032   6.13564605]
 [  8.00737302  -9.76891849 -19.23974289  11.3271237    7.38205098]
 [ 20.28868505 -17.5096973    8.01065341  -8.62126282 -18.07411016]
 [ -6.31576968 -17.31202523 -13.97937646 -13.99465532  -3.17189743]
 [  4.22183304  -1.05841052 -11.25461884 -11.60744667   8.52949869]]
B[i+1], before projection:
 23.923677243129422

B: 23.923677243129422
B_max_wrt_tau_feas: 11.365130922344461
B


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 4025382.9881178304
obj_E: 5694.76858413384
obj_R: 875.6824484704216
obj_I: 803674.372638383

welfare_perturbed: 4084519.9779817355
obj_E_perturbed: 5628.302688929322
obj_R_perturbed: 875.9808960407146
obj_I_perturbed: 815528.6554168161

tau[:, :, i+1], before projection:
 [[ -8.39852512 -28.2877813   11.54790894 -10.70582583  18.24775618]
 [ -8.70900289 -44.52291402  -1.71638393   6.60961934  -4.54487561]
 [ 10.65234472  -0.72289945  22.01030671  27.27378145   2.52820788]
 [  3.4661378   -8.13258737  -0.079348    -9.1885338    6.37275665]
 [ 18.35821934  -1.60512113 -17.28125582   8.41550631 -17.5248378 ]
 [-10.66644423  -1.32410342  10.83312518 -11.62333794  -5.92440433]
 [ 16.74336516 -20.88432691 -15.96684114 -17.9133413    7.96870942]]
B[i+1], before projection:
 -25.638645571954843

B: -25.638645571954843
B_max_wrt_tau_feas: 10.779359367111859



prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3692989.2825615807
obj_E: 6235.86319152254
obj_R: 5520.023420436773
obj_I: 741623.534656144

welfare_perturbed: 3677446.8435321385
obj_E_perturbed: 6235.895221990224
obj_R_perturbed: 5520.58186116442
obj_I_perturbed: 738515.592478796

tau[:, :, i+1], before projection:
 [[-3.66281836 -6.44083775  3.04105187 -8.72797836  0.75917042]
 [-0.67136007 -3.85254977 -6.07145382  0.74342915  0.17334261]
 [-3.66286591  3.2317112  -2.63263984  4.51026834 -5.31509207]
 [-1.26990409  3.27743522  4.83811469  2.52060113  0.66647817]
 [-3.50438739  3.11420197 -2.3002479   0.08309918  9.25495048]
 [ 3.66953949 -0.56944222 -7.82382556  3.19826819  6.29938691]
 [-4.90760835 -4.17105616  6.34273787  2.70284925  0.88861218]]
B[i+1], before projection:
 0.15732214816661325

B: 0.15732214816661325
B_max_wrt_tau_feas: 11.333593690216473
B_feas: 0.15732214816661325

tau[:, :


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3795826.5832700464
obj_E: 6127.2663351044375
obj_R: 8756.943234954228
obj_I: 765471.3533549218

welfare_perturbed: 3814203.7725365567
obj_E_perturbed: 6120.394354259658
obj_R_perturbed: 8756.959228076004
obj_I_perturbed: 769149.5559936834

tau[:, :, i+1], before projection:
 [[ -4.22412535   1.71165324   3.84187606  -1.95595895  -7.48116896]
 [  2.6809987    1.09731422  -2.17747019  -3.86826923  -2.98297292]
 [-10.47464521   3.21848448  -1.59092599  -9.12916681   6.9266559 ]
 [ -3.24970588   4.07307379   4.88447991   0.55086664  -0.94048177]
 [  7.18583012   5.80861269  -6.27239368  -9.40447468  -4.84352826]
 [  5.01564875  -0.31957761  -2.2519667    6.26820525   4.43428508]
 [  4.77953544   0.51747545  -0.30974647   6.13092597   1.82851892]]
B[i+1], before projection:
 -6.122322152908511

B: -6.122322152908511
B_max_wrt_tau_feas: 10.401141237478301


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3737641.4144678223
obj_E: 6163.967150867282
obj_R: 10617.958902466404
obj_I: 755680.6549356839

welfare_perturbed: 3754711.968792561
obj_E_perturbed: 6181.754029261616
obj_R_perturbed: 10618.306152489391
obj_I_perturbed: 759087.998299297

tau[:, :, i+1], before projection:
 [[ -7.35042941   1.78787042   4.19951807  -5.56405215  -3.82977055]
 [ -7.71013374   0.44702175   4.55175192  -4.8271548    6.10734185]
 [  1.86459901   2.02138746   6.18272144  -4.33892325  -1.44095211]
 [ -1.57669656  -5.11784009  -0.92967798  -1.32396663   2.87437201]
 [  2.57439205  -3.86904624   4.96162381  -2.11182561   3.84471939]
 [  7.25563296   5.34155747  -0.9259211    2.19157487 -13.16875079]
 [  4.57503766   0.80554891  -0.35925358   5.64390531   3.7366813 ]]
B[i+1], before projection:
 2.0751699906866223

B: 2.0751699906866223
B_max_wrt_tau_feas: 11.675841592757813


In [ ]:
# dict_latency_params["Belmont"]

In [324]:
min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

# print("argmin_B:", argmin_B)
# print("B[argmin_welfare_list]:", B[argmin_welfare_list])
# print("min_welfare:", min_welfare)
# print()
# print(welfare_list)

# B
print()
print(min_welfare)
print()
print(argmin_welfare_list)
print()
print("len(welfare_list)", len(welfare_list))
print()
print("argmin_tau", argmin_tau)
print()
print("argmin_B", argmin_B)

# welfare_list[argmin_welfare_list]
# B[argmin_welfare_list]


3602424.493577951

115

len(welfare_list) 199

argmin_tau [[0.2269 0.3194 0.2683 0.     0.3194]
 [0.     0.2498 0.2498 0.0882 0.1051]
 [0.9995 0.     0.8429 0.8238 0.3535]
 [0.0908 0.     0.     0.0541 0.3029]
 [0.053  0.1608 0.0399 1.6043 1.2384]
 [0.1922 0.1281 0.1922 0.192  0.    ]
 [0.     0.0806 0.     0.0947 0.    ]]

argmin_B 0.0


In [331]:
# np.round(B, decimals=3)

In [332]:
# index_next_init = argmin_welfare_list
# B_next_init = B[index_next_init]
# tau_next_init = tau[:, :, index_next_init]

# print("index_next_init:", index_next_init)
# print("B_next_init:", B_next_init)
# print("tau_next_init:", tau_next_init)


In [333]:
# lambda combinations to try to simulate:
# (1, 1, 1)
# (1, 1, 5)
# (1, 1, 10)
# (1, 5, 5)
# (1, 2, 1)
# (1, 2, 5)

# (2, 2, 1)
# (2, 2, 5)
# (2, 5, 5)
# (2, 5, 10)

In [ ]:
## Initialize lambda:

lambda_E, lambda_R, lambda_I = 2.0, 5.0, 5.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

directory_inits = '../data/opt_tolls_subsidies_metrics/'
df_inits = pd.read_csv(directory_inits + 'opt_CBCP___' + filename_segment + '.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print(argmin_tau)
print()
print(argmin_B)


In [334]:
# argmin_tau = np.array([[0.0, 0.3194, 0.3194, 0.0, 0.0], \
#                        [0.2498, 0.2498, 0.2498, 0.0, 0.0], \
#                        [0.0, 0.0, 0.0, 0.9995, 0.9995], \
#                        [0.0, 1.0281, 1.0281, 1.0281, 0.0], \
#                        [1.6043, 0.0, 0.0, 1.6043, 0.0], \
#                        [0.0, 0.1922, 0.1922, 0.1922, 0.0], \
#                        [0.0, 0.0, 0.0, 0.2178, 0.2178]])

# argmin_B = 10.6925

In [335]:
argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                    argmin_tau, argmin_B, od_to_edges_array, \
                                    demand_array_temp, VoT_array, num_el, coeff_input)

print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)
print("min_welfare:", min_welfare)
print()
print("argmin_y:\n")

argmin_y



prob.status: optimal
argmin_tau: [[0.2269 0.3194 0.2683 0.     0.3194]
 [0.     0.2498 0.2498 0.0882 0.1051]
 [0.9995 0.     0.8429 0.8238 0.3535]
 [0.0908 0.     0.     0.0541 0.3029]
 [0.053  0.1608 0.0399 1.6043 1.2384]
 [0.1922 0.1281 0.1922 0.192  0.    ]
 [0.     0.0806 0.     0.0947 0.    ]]
argmin_B: 0.0
min_welfare: 3602424.493577951

argmin_y:



{(0, 0, 0, 0, 0): -0.0,
 (0, 0, 0, 1, 0): -0.0,
 (0, 0, 0, 2, 0): 47.77809012103125,
 (0, 1, 0, 0, 0): -0.0,
 (0, 1, 0, 1, 0): 3.380754251641479e-05,
 (0, 1, 0, 2, 0): 27.56424895459089,
 (0, 2, 0, 0, 0): 8.425127481952788e-05,
 (0, 2, 0, 1, 0): 115.1573637327492,
 (0, 3, 0, 0, 0): 0.00025428051134073035,
 (0, 3, 0, 1, 0): 134.7584614454742,
 (0, 4, 0, 0, 0): 0.0032578160844850533,
 (0, 4, 0, 1, 0): 287.27782252703923,
 (0, 0, 0, 0, 1): -0.0,
 (0, 0, 0, 1, 1): -0.0,
 (0, 0, 0, 2, 1): 47.77809012103125,
 (0, 1, 0, 0, 1): -0.0,
 (0, 1, 0, 1, 1): 2.3922376236025393e-05,
 (0, 1, 0, 2, 1): 27.56425883975717,
 (0, 2, 0, 0, 1): 8.507348579692664e-05,
 (0, 2, 0, 1, 1): 115.15736291053823,
 (0, 3, 0, 0, 1): 0.00015394962044690011,
 (0, 3, 0, 1, 1): 134.7585617763651,
 (0, 4, 0, 0, 1): 0.0018704969445164376,
 (0, 4, 0, 1, 1): 287.27920984617924,
 (0, 0, 0, 0, 2): -0.0,
 (0, 0, 0, 1, 2): -0.0,
 (0, 0, 0, 2, 2): 47.77809012103125,
 (0, 1, 0, 0, 2): -0.0,
 (0, 1, 0, 1, 2): 2.8629388357970206e-05,
 

In [336]:
# demand_array_temp

# tau_upper_limit

# B_upper_limit

# y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

# T
# num_edges
# num_gp_lanes
# B_perturbed[i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input

# np.minimum(np.array([1, 2]), 0.5)

# tau_upper_limit
# argmin_welfare_list

# Compute Cost Metrics

In [337]:
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

argmin_x = {}
for e in range(num_edges):
    for t in range(T):
        argmin_x[(e, 0, t)] = 0
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_x[(e, 1, t)] = 0
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 2, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)

argmin_y_in_el_total = {}      
for e in range(num_edges):
    for t in range(T):
        argmin_y_in_el_total[(e, 0, t, 'el')] = sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'el')] = sum(argmin_y[(od, g, e, 2, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 0, t, 'in')] = sum(argmin_y[(od, g, e, 0, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'in')] = sum(argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)


# y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indice

# argmin_x

argmin_y_in_el_total

{(0, 0, 0, 'el'): 4.401920590597546e-05,
 (0, 1, 0, 'el'): 596.3167027772606,
 (0, 0, 0, 'in'): 874.2626414471843,
 (0, 1, 0, 'in'): 3377.524244247947,
 (0, 0, 1, 'el'): 3.1197011218414645e-05,
 (0, 1, 1, 'el'): 596.3167155994553,
 (0, 0, 1, 'in'): 528.550885135476,
 (0, 1, 1, 'in'): 3723.2360005596547,
 (0, 0, 2, 'el'): 3.728347518373317e-05,
 (0, 1, 2, 'el'): 596.3167095129913,
 (0, 0, 2, 'in'): 693.440549217239,
 (0, 1, 2, 'in'): 3558.346336477892,
 (0, 0, 3, 'el'): 0.0,
 (0, 1, 3, 'el'): 596.3167467964665,
 (0, 0, 3, 'in'): 1002.0023126506655,
 (0, 1, 3, 'in'): 3249.784573044465,
 (0, 0, 4, 'el'): 3.1225580966849444e-05,
 (0, 1, 4, 'el'): 596.3167155708855,
 (0, 0, 4, 'in'): 478.26444521883974,
 (0, 1, 4, 'in'): 3773.5224404762916,
 (1, 0, 0, 'el'): 0.0,
 (1, 1, 0, 'el'): 658.9583981680674,
 (1, 0, 0, 'in'): 1118.3320863827685,
 (1, 1, 0, 'in'): 3396.9403626695353,
 (1, 0, 1, 'el'): 7.97790532046241e-05,
 (1, 1, 1, 'el'): 658.9583183890143,
 (1, 0, 1, 'in'): 354.35998645513416,
 (1

In [338]:
# coeff_input

print("argmin_tau[5, 3]:", argmin_tau[5, 3])
print("argmin_x[(5, 0, 3)]:", argmin_x[(5, 0, 3)])
print("argmin_x[(5, 1, 3)]:", argmin_x[(5, 1, 3)])


argmin_tau[5, 3]: 0.192
argmin_x[(5, 0, 3)]: 873.0967628430186
argmin_x[(5, 1, 3)]: 5771.9739604126735


In [339]:
travel_times = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times[(e, 0, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times[(e, 1, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

edge_demand = {}
avg_travel_time = {}
percent_on_express = {}
obj_E = np.zeros(num_edges)
obj_I = np.zeros(num_edges)
obj_R = np.zeros(num_edges)
obj = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time[e, 'el'] = 0.0
    avg_travel_time[e, 'in'] = 0.0
    avg_travel_time[e, 'ex'] = 0.0
    avg_travel_time[e, 'gp'] = 0.0
    
    percent_on_express[e, 'el'] = 0.0
    percent_on_express[e, 'in'] = 0.0
    percent_on_express[e, 'all'] = 0.0
    
    obj_E[e] = 0.0
    obj_I[e] = 0.0
    obj_R[e] = 0.0
    obj[e] = 0.0
    
    for t in range(T):
        edge_demand[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))
    percent_on_express[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))
    percent_on_express[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] + edge_demand[e, t, 'in'] for t in range(T))
    
    avg_travel_time[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))    
    avg_travel_time[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))    
#     avg_travel_time[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time[e, 'ex'] += sum(travel_times[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time[e, 'gp'] += sum(travel_times[(e, 1, t)] for t in range(T)) / T 
    
    obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

    obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

    obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time
# percent_on_express
# obj_R



In [340]:
# e = 0
# od = 0
# g = 0
# t = 0

# # sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
# #                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + tau[e, t]) \
#     for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# #                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
# #                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) ) 

# # argmin_y[(od, g, e, 1, t)]



# # sum(edge_demand[e, t, 'el'] for t in range(T) )

In [341]:
# travel_times
argmin_x

{(0, 0, 0): 874.2626854663902,
 (0, 1, 0): 3973.8409470252077,
 (0, 0, 1): 528.5509163324873,
 (0, 1, 1): 4319.55271615911,
 (0, 0, 2): 693.4405865007142,
 (0, 1, 2): 4154.663045990883,
 (0, 0, 3): 1002.0023126506655,
 (0, 1, 3): 3846.101319840932,
 (0, 0, 4): 478.2644764444207,
 (0, 1, 4): 4369.839156047177,
 (1, 0, 0): 1118.3320863827685,
 (1, 1, 0): 4055.898760837603,
 (1, 0, 1): 354.36006623418734,
 (1, 1, 1): 4819.870780986184,
 (1, 0, 2): 324.7941192482156,
 (1, 1, 2): 4849.436727972156,
 (1, 0, 3): 1044.955085364953,
 (1, 1, 3): 4129.27576185542,
 (1, 0, 4): 1036.849254106847,
 (1, 1, 4): 4137.381593113525,
 (2, 0, 0): 907.6147693997235,
 (2, 1, 0): 5886.103160785078,
 (2, 0, 1): 1208.087287825698,
 (2, 1, 1): 5585.630642359102,
 (2, 0, 2): 947.3662648469067,
 (2, 1, 2): 5846.351665337894,
 (2, 0, 3): 951.8412740065185,
 (2, 1, 3): 5841.876656178283,
 (2, 0, 4): 1105.5917980930035,
 (2, 1, 4): 5688.1261320918,
 (3, 0, 0): 1350.336734431026,
 (3, 1, 0): 4697.301564669474,
 (3, 0,

In [342]:
# print("argmin_x[(3, 0, 0)]:", argmin_x[(3, 0, 0)])
# print("argmin_x[(3, 1, 0)]:", argmin_x[(3, 1, 0)])

In [343]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# argmin_B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_time[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_time[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_E[e] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_I[e] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_R[e] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [344]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save



,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.23,0.32,0.27,0.00,0.32,0.23,0.0,14.75,0.0,16.82,1.35,3.89,669.01,83177.54,706.00,413695.71
e=2,0.00,0.25,0.25,0.09,0.11,0.14,0.0,14.99,0.0,17.18,2.24,4.20,845.96,91593.07,370.79,457803.34
e=3,1.00,0.00,0.84,0.82,0.35,0.60,0.0,15.07,0.0,17.21,5.12,12.54,3379.45,342110.12,2880.65,1702906.26
e=4,0.09,0.00,0.00,0.05,0.30,0.09,0.0,22.34,0.0,25.46,1.34,8.02,1934.00,156190.93,590.13,781872.00
e=5,0.05,0.16,0.04,1.60,1.24,0.62,0.0,16.06,0.0,18.24,6.06,17.64,5524.18,511518.51,3455.70,2551362.39
e=6,0.19,0.13,0.19,0.19,0.00,0.14,0.0,14.26,0.0,16.11,1.53,2.99,752.76,81556.52,623.10,406172.62
e=7,0.00,0.08,0.00,0.09,0.00,0.04,0.0,16.42,0.0,18.55,2.49,4.02,1094.77,121362.77,190.20,608052.41


In [345]:
# # e = 0
# # t = 4

# e = 4
# t = 3

# print("travel_times[(e, 0, t)]:\n", travel_times[(e, 0, t)])
# print()
# print("travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]):\n", \
#       travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]) )
# print()
# print("travel_times[(e, 1, t)]:\n", travel_times[(e, 1, t)])
# print()
# print("Total ineligible user express lane flow at (e, t):\n", argmin_y_in_el_total[(e, 0, t, "in")])

# VoT_array[:, in_indices, t]
# # demand_array[:, in_indices]

# # print("")

# obj_R

# for e in range(num_edges):
#     for t in range(T):
#         print(sum(argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices) \
#                     + sum(argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in el_indices) )

# VoT_array[:, :, 0]

In [346]:
random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "opt_CBCP___" + random_string + '.csv'

df_opt_save.to_csv(directory_to_save + random_filename)

# Compute good initializations for alpha:

In [347]:
# # el_indices
# # in_indices

# e = 0
# # VoT_array[edge_to_od_dict[e], in_indices, :]

# VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :]
# VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices, :]

# # print("VoT_array[:, in_indices, :]:\n", VoT_array[:, in_indices, :])
# # print()
# # print("VoT_array[edge_to_od_dict[e], :, :]:\n", VoT_array[edge_to_od_dict[e], :, :])

# # VoT_array_trunc_in

# demand_array.shape

In [348]:
# arr_temp = np.stack([np.array([1, 2, 3, 4, 5]), np.array([3, 1, 4, 5, 2]), np.array([2, 5, 1, 4, 3])]).T
# print(arr_temp)
# # arr_temp[0:, :]
# # print(np.sum(arr_temp))

# # arr_temp[[2, 3], [1, 2]]
# arr_temp[np.ix_([2,3], [1,2])]

In [349]:
alpha_init = np.zeros((num_edges, T))

VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
#     print("edge_to_od_dict[e]:", edge_to_od_dict[e])
    
    for t in range(T):
        
        print()
        print("e:", e)
        print("t:", t)
        
        ## Ineligible users:
        
        VoT_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len (in_indices)))
        demand_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len (in_indices)))
        
#         for od_index, od_temp in enumerate(edge_to_od_dict[e]):
#             for g_index, g_temp in enumerate(in_indices):
#                 VoT_array_trunc_in[od_index, g_index] = VoT_array[od_temp, g_temp, t] 
#                 demand_array_trunc_in[od_index, g_index] = demand_array[od_temp, g_temp] 
        
#         VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :, t]
#         VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices]
#         demand_array_trunc_in_temp = demand_array[edge_to_od_dict[e], :]
#         demand_array_trunc_in = demand_array_trunc_in_temp[:, in_indices]
        
        VoT_array_trunc_in = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], in_indices)]
        demand_array_trunc_in = demand_array[np.ix_(edge_to_od_dict[e], in_indices)]

#         print("np.sum(demand_array_trunc_in):", np.sum(demand_array_trunc_in))
#         print("(Same sum computed directly):", sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                     for g in in_indices]))

        assert np.abs(np.sum(demand_array_trunc_in) - sum([demand_array[od, g] for od in edge_to_od_dict[e] \
                                                    for g in in_indices])) <= 1E-3
#         assert np.sum(demand_array_trunc_in_flattened) == sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                             for g in in_indices])
        
#         print("VoT_array_trunc_in.shape:", VoT_array_trunc_in.shape)
#         print("demand_array_trunc_in.shape:", demand_array_trunc_in.shape)
#         print("len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])", \
#               len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]))
#         print("demand_array_trunc_in:", demand_array_trunc_in)
#         print("[demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]:", [demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])

        VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
        demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
        VoT_demand_in = np.stack([VoT_array_trunc_in_flattened, demand_array_trunc_in_flattened]).T    
        VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
        VoT_demand_in_cumul = np.zeros(VoT_demand_in_sorted.shape)
        VoT_demand_in_cumul[:, 0] = VoT_demand_in_sorted[:, 0]
        for row_index in range(VoT_demand_in_cumul.shape[0]):
            VoT_demand_in_cumul[row_index, 1] = np.sum(VoT_demand_in_sorted[row_index:, 1])
        
    
        
        assert np.abs(np.sum(VoT_demand_in[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                            for g in in_indices)) <= 1E-3
        assert np.abs(np.sum(VoT_demand_in_sorted[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                            for g in in_indices)) <= 1E-3
        assert np.abs(VoT_demand_in_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in in_indices)) <= 1E-3

        ## Eligible users:

        # Flatten VoT_array_trunc_in and demand_array_trunc_in, and concatenate.
        # Sort by VoT
        # Create cumulative demand array

#         VoT_array_trunc_el_temp = VoT_array[edge_to_od_dict[e], :, t]
#         VoT_array_trunc_el = VoT_array_trunc_el_temp[:, el_indices]
#         demand_array_trunc_el_temp = demand_array[edge_to_od_dict[e], :]
#         demand_array_trunc_el = demand_array_trunc_el_temp[:, el_indices]

        VoT_array_trunc_el = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices)]
        demand_array_trunc_el = demand_array[np.ix_(edge_to_od_dict[e], el_indices)]

        VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
        demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
        VoT_demand_el = np.stack([VoT_array_trunc_el_flattened, demand_array_trunc_el_flattened]).T    
        VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
        VoT_demand_el_cumul = np.zeros(VoT_demand_el_sorted.shape)
        VoT_demand_el_cumul[:, 0] = VoT_demand_el_sorted[:, 0]
        for row_index in range(VoT_demand_el_cumul.shape[0]):
            VoT_demand_el_cumul[row_index, 1] = np.sum(VoT_demand_el_sorted[row_index:, 1])
    
        print("VoT_demand_el_cumul[0, 1]:", VoT_demand_el_cumul[0, 1])
        print("(Same sum computed directly):", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices))
        print()
        assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices)) <= 1E-3
        
        ## Ineligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        VoT_in_boundary_index = max([row_index for row_index in range(VoT_demand_in_cumul.shape[0]) \
                                     if VoT_demand_in_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'in')]])
        VoT_in_boundary[e, t] = VoT_demand_in_cumul[VoT_in_boundary_index, 0]
        
        ## Eligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        print("Total el user flow at e, t:", \
                 sum(demand_array[(od, g)] for od in edge_to_od_dict[e] for g in el_indices))
        print("argmin_y_in_el_total[(e, 0, t, 'el')]]:", argmin_y_in_el_total[(e, 0, t, 'el')])
        print("VoT_demand_el_sorted[:, 1]", VoT_demand_el_sorted[:, 1])
        print("VoT_demand_el_cumul[:, 1]", VoT_demand_el_cumul[:, 1])
        print("VoT_demand_el_cumul[0, 1]:", VoT_demand_el_cumul[0, 1])
        print("(Same sum computed directly):", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices))
        print()
        assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                        for g in el_indices)) <= 1E-3
        
        VoT_el_boundary_index = max([row_index for row_index in range(VoT_demand_el_cumul.shape[0]) \
                                     if VoT_demand_el_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'el')] - 1E-3])
        VoT_el_boundary[e, t] = VoT_demand_el_cumul[VoT_el_boundary_index, 0]
        
        if argmin_y_in_el_total[(e, 0, t, 'el')] <= 1E-3:
            alpha_init[e, t] = 0
        else:
            alpha_init[e, t] = 1.0 - VoT_el_boundary[e, t]/VoT_in_boundary[e, t]
        
        print("VoT_el_boundary_index:", VoT_el_boundary_index)
        print("VoT_in_boundary_index:", VoT_in_boundary_index)
        print("VoT_el_boundary[e, t]:", VoT_el_boundary[e, t])
        print("VoT_in_boundary[e, t]:", VoT_in_boundary[e, t])
        print("alpha_init[e, t]:", alpha_init[e, t])
        
        assert 0.0 <= alpha_init[e, t] <= 1.0



e: 0
t: 0
VoT_demand_el_cumul[0, 1]: 596.3167467964665
(Same sum computed directly): 596.3167467964665

Total el user flow at e, t: 596.3167467964665
argmin_y_in_el_total[(e, 0, t, 'el')]]: 4.401920590597546e-05
VoT_demand_el_sorted[:, 1] [ 47.77809012   7.64672375  88.76462936  19.95836002  75.66495318
  18.9161492  119.4231777   27.56428276   4.41157139  51.21036309
  11.51443847  43.65285761  10.913163    68.89798713]
VoT_demand_el_cumul[:, 1] [596.3167468  548.53865668 540.89193293 452.12730357 432.16894355
 356.50399037 337.58784116 218.16466346 190.6003807  186.18880931
 134.97844622 123.46400774  79.81115014  68.89798713]
VoT_demand_el_cumul[0, 1]: 596.3167467964665
(Same sum computed directly): 596.3167467964665

VoT_el_boundary_index: 13
VoT_in_boundary_index: 18
VoT_el_boundary[e, t]: 0.10238603988603988
VoT_in_boundary[e, t]: 2.042296664152623
alpha_init[e, t]: 0.0

e: 0
t: 1
VoT_demand_el_cumul[0, 1]: 596.3167467964665
(Same sum computed directly): 596.3167467964665

Total

In [350]:
# # VoT_array_trunc_in_flattened
# # demand_in_trunc_in_flattened

# print("argmin_y_in_el_total[(e, 0, t, el)]:\n", argmin_y_in_el_total[(e, 0, t, 'el')])
# print()
# print("argmin_y_in_el_total[(e, 0, t, in)]:\n", argmin_y_in_el_total[(e, 0, t, 'in')])
# print()
# print("VoT_demand_el_cumul:\n", VoT_demand_el_cumul)
# print()
# print("VoT_demand_in_cumul:\n", VoT_demand_in_cumul)



In [351]:
# argmin_tau
# alpha_init

init_tau_alpha_array = np.block([argmin_tau, alpha_init])
init_tau_alpha_array

array([[0.2269    , 0.3194    , 0.2683    , 0.        , 0.3194    ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.2498    , 0.2498    , 0.0882    , 0.1051    ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.9995    , 0.        , 0.8429    , 0.8238    , 0.3535    ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.0908    , 0.        , 0.        , 0.0541    , 0.3029    ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.053     , 0.1608    , 0.0399    , 1.6043    , 1.2384    ,
        0.        , 0.        , 0.93613372, 0.        , 0.        ],
       [0.1922    , 0.1281    , 0.1922    , 0.192     , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.0806    , 0.        , 0.0947    , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [352]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# df_inits_save

random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "inits___" + random_string + '.csv'

df_inits_save.to_csv(directory_to_save + random_filename)

In [284]:
# # argmin_y_in_el_total[(e, 0, t, 'in')]
# # demand_array_trunc_in_flattened

# for e in range(num_edges):
#     for t in range(T):
#         print("e:", e)
#         print("t:", t)
#         print("argmin_y_in_el_total[(e, 0, t, 'el')]:", argmin_y_in_el_total[(e, 0, t, 'el')])
#         print()

# Testing solve CBCP direct with a small example:

In [ ]:
# od_to_edges_array = np.array([[0, 0], [0, 1], [1, 1], [1, 2]])
# od_to_edges_list_full = [[0], [0, 1], [1], [1, 2]]
# edge_to_ods = [[0, 1], [1, 2, 3], [3]]

# num_od = 3
# num_edges = 3
# num_gp_lanes = 3
# num_groups = 5
# T = 5
# el_indices = [0, 1]
# in_indices = [2, 3, 4]
# B = 3
# tau = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
#         tau[e, t] = 1.5 + 0.5 * e + 0.2 * t

# demand_array = np.zeros((num_od, num_groups))
# VoT_array = np.zeros((num_od, num_groups, T))

# for od in range(num_od):
#     for g in range(num_groups):
#         demand_array[od, g] = 1.0 + od + 0.1 * g
#         VoT_array[od, g, t] = 2.0 + od + 0.1 * g
        
# coeff_input = np.array([1, 1, 0, 0, 0])
# latency_params_length = coeff_input.shape[0]

# ex_to_gp_multiplier = np.array([1/num_gp_lanes**p for p in range(latency_params_length)]).reshape((latency_params_length, 1)) \
#                         @ np.ones((1, num_edges))
# a = np.zeros((latency_params_length, num_edges, 2))
# coeff[:, :, 0] = coeff_input.reshape((latency_params_length, 1)) @ np.ones((1, num_edges))
# coeff[:, :, 1] = coeff[:, :, 0] * ex_to_gp_multiplier


# y_vals = solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                            demand_array, VoT_array, num_el, coeff_input)


# # y = {}
# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)

# # x = {}
# # for e in range(num_edges):
# #     for k in [0, 1]:
# #         for t in range(T):
# #             x[(e, k, t)] = cp.Variable(1)

# # # Objective:
# # func = 0.0
# # for e in range(num_edges):

# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
# #             for g in in_indices:
# #                 func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

# # objective = cp.Minimize(func)

# # # Constraints:
# # constraints = []

# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in el_indices  \
# #                 for k in [0, 1, 2] for t in range(T)]
# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in in_indices  \
# #                 for k in [0, 1] for t in range(T)]


# # for e in range(num_edges):
# #     for t in range(T):
# # #         print("e:", e)
# # #         print("edge_to_ods[e]:", edge_to_ods[e])

# #         ## Edge contraints:
# #         constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 0, t)] ]
# #         constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 1, t)] ]

# #         ## Group flow constraints:
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in el_indices]
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in in_indices]

# # constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
# #                 <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]

# # # Problem:
# # prob = cp.Problem(objective, constraints)

# # # Solve:
# # result = prob.solve()


# # assert prob.status != "infeasible", "problem.status should not be infeasible."
# # assert prob.status != "unbounded", "problem.status should not be unbounded."
# # print()
# # print("prob.status:", prob.status)

# # # Extract Values:
# # y_values = {}
# # for e in range(num_edges):
# #     for od in edge_to_ods[e]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)




In [ ]:
# demand_edges_array

In [ ]:
# np.sum(demand_array[3:16, :])/5

In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
